In [14]:
import pandas as pd 
import numpy as np 
import re 

In [15]:
df_pokemon = pd.read_csv('pokemon.csv')

df_pokemon_tipos_1 = pd.read_csv('pokemon_tipos_1.csv')

df_pokemon_tipos_2 = pd.read_csv('pokemon_tipos_2.csv')

In [16]:
# Por ser um a base de dados pequena, optei por trabalhar com todos os dados disponibilizados, para isso, me DF principal será a junção dos 3.

df_pokemon_complete = df_pokemon.merge(df_pokemon_tipos_1).merge(df_pokemon_tipos_2)

In [17]:
# Exibindo informações úteis antes do tratamento dos dados. O info() será útil para analizar os tratamentos necessarios e o columns para saber o nome das colunas.

df_pokemon_complete.info()

df_pokemon_complete.columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 0 to 413
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   #                        414 non-null    int64  
 1   pokemon_nome             414 non-null    object 
 2   pokemon_tipo_1_id        414 non-null    int64  
 3   pokemon_tipo_2_id        414 non-null    float64
 4   pokemon_total            414 non-null    int64  
 5   pokemon_hp               414 non-null    int64  
 6   pokemon_attack           414 non-null    int64  
 7   pokemon_defense          414 non-null    int64  
 8   pokemon_ataque_especial  414 non-null    int64  
 9   pokemon_defesa_especial  414 non-null    int64  
 10  pokemon_velocidade       414 non-null    int64  
 11  pokemon_geracao          414 non-null    int64  
 12  pokemon_lendario         414 non-null    bool   
 13  pokemon_tipo_1_nome      414 non-null    object 
 14  pokemon_tipo_2_nome      4

Index(['#', 'pokemon_nome', 'pokemon_tipo_1_id', 'pokemon_tipo_2_id',
       'pokemon_total', 'pokemon_hp', 'pokemon_attack', 'pokemon_defense',
       'pokemon_ataque_especial', 'pokemon_defesa_especial',
       'pokemon_velocidade', 'pokemon_geracao', 'pokemon_lendario',
       'pokemon_tipo_1_nome', 'pokemon_tipo_2_nome'],
      dtype='object')

In [18]:
# Para visualizar a tabela e o que são, na pratica, os dados, colunas e informações do DF antes do tratamento.

df_pokemon_complete.head(1)

,#,pokemon_nome,pokemon_tipo_1_id,pokemon_tipo_2_id,pokemon_total,pokemon_hp,pokemon_attack,pokemon_defense,pokemon_ataque_especial,pokemon_defesa_especial,pokemon_velocidade,pokemon_geracao,pokemon_lendario,pokemon_tipo_1_nome,pokemon_tipo_2_nome
0,1,Bulbasaur,10,14.0,318,45,49,49,65,65,45,1,False,Grass,Poison


##### 1) Verifique se há a necessidade de realizar algum tratamento na base de dados, caso julgue necessário realize o tratamento e comente no código o porquê de estar realizando determinada técnica de limpeza.

In [19]:
df_pokemon_complete.drop(['#'], axis=1, inplace=True)
# Identifiquei que a coluna "#" não me retornava nenhuma informação passivel de analise. sua permanencia poderia mascarar dados duplicados, por isso exclui a coluna.

df_pokemon_complete['pokemon_tipo_2_id'] = df_pokemon_complete['pokemon_tipo_2_id'].astype(np.int16)
# Para facilitar as analises, transformei a coluna float64 em int16. Se fosse necessario,  poderia aplicar
# essa mesma técnica em todas as colunas int64, mas vendo que a tabela é pequena, +- 49 KB, entendo não ser necessário, 
# se ela tivesse MBs ou apresentasse lentidão na execusão, faria essa alteração.

df_pokemon_complete.drop_duplicates(keep='first', inplace=True)
# Não tenho interesse em realizar as analises com dados duplicados, dessa forma já realizo essa busca e limpeza dos duplicates.


In [20]:
#Exibindo informações úteis após o tratamento. O info será útil para identificar e visualizar as alterações e o columns para saber o nome das colunas.

df_pokemon_complete.info()

df_pokemon_complete.columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 0 to 413
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   pokemon_nome             414 non-null    object
 1   pokemon_tipo_1_id        414 non-null    int64 
 2   pokemon_tipo_2_id        414 non-null    int16 
 3   pokemon_total            414 non-null    int64 
 4   pokemon_hp               414 non-null    int64 
 5   pokemon_attack           414 non-null    int64 
 6   pokemon_defense          414 non-null    int64 
 7   pokemon_ataque_especial  414 non-null    int64 
 8   pokemon_defesa_especial  414 non-null    int64 
 9   pokemon_velocidade       414 non-null    int64 
 10  pokemon_geracao          414 non-null    int64 
 11  pokemon_lendario         414 non-null    bool  
 12  pokemon_tipo_1_nome      414 non-null    object
 13  pokemon_tipo_2_nome      414 non-null    object
dtypes: bool(1), int16(1), int64(9), object(3)


Index(['pokemon_nome', 'pokemon_tipo_1_id', 'pokemon_tipo_2_id',
       'pokemon_total', 'pokemon_hp', 'pokemon_attack', 'pokemon_defense',
       'pokemon_ataque_especial', 'pokemon_defesa_especial',
       'pokemon_velocidade', 'pokemon_geracao', 'pokemon_lendario',
       'pokemon_tipo_1_nome', 'pokemon_tipo_2_nome'],
      dtype='object')

In [21]:
# Para visualizar a tabela e o que são, na pratica, os dados, colunas e informações do DF após o tratamento.
df_pokemon_complete.head(1)

,pokemon_nome,pokemon_tipo_1_id,pokemon_tipo_2_id,pokemon_total,pokemon_hp,pokemon_attack,pokemon_defense,pokemon_ataque_especial,pokemon_defesa_especial,pokemon_velocidade,pokemon_geracao,pokemon_lendario,pokemon_tipo_1_nome,pokemon_tipo_2_nome
0,Bulbasaur,10,14,318,45,49,49,65,65,45,1,False,Grass,Poison


##### 2) Procure os pokémons do tipo Grass que possuem mais de 100 de ataque e são da geração 3.

In [22]:
# Estudei os dados, logo, sabendo a relação entre o tipo_1_id e o tipo_1_nome, irei farei a busca pelos ID, 
# com intuito de agilizar o processamento e usufruir das informações disponiveis. Grass == 10
# Filtrei os pokémons utilizando o query(), depois apresentei o resultado, utilizei a função to_string(index = False) para remover o index da saída.

df_pokemons_grass_attack_100_gen_3 = df_pokemon_complete.query('pokemon_tipo_1_id == 10 & pokemon_attack > 100 & pokemon_geracao == 3')

print(f"Os pokémons do tipo Grass, que possuem mais de 100 de ataque e são da geração 3: \n")

print(df_pokemons_grass_attack_100_gen_3[['pokemon_nome','pokemon_tipo_1_nome','pokemon_attack']].to_string(index = False))

Os pokémons do tipo Grass, que possuem mais de 100 de ataque e são da geração 3: 

         pokemon_nome pokemon_tipo_1_nome  pokemon_attack
SceptileMega Sceptile               Grass             110
             Cacturne               Grass             115
              Breloom               Grass             130


##### 3) Faça um comparativo entre os tipos de pokémon e as gerações, evidencie em cada tipo quais gerações possuem mais de 50 pokémons. Dica: use pivoteamento.

In [23]:
# Criei a função more_fidty_pokemons, que, ao utilizada no aggfunc, recebpokemons_tipe_fire_grass e os valores da coluna informada. Ela realiza a soma e retorna os valores que são maiores de 50.
# o pokemons_tipe_and_gen é um pivot_table, que traz uma tabela de fácil visualização, evidenciando os tipos e as gerações de pokémons que possuem mais de 50 individuos.

def more_fifty_pokemons(values):
    sum_values = values.sum()
    if sum_values > 50:
        return sum_values

df_pokemons_tipe_and_gen = pd.pivot_table(df_pokemon_complete,
    index=['pokemon_geracao'],  
    columns=['pokemon_tipo_1_nome'],
    aggfunc={'pokemon_geracao': more_fifty_pokemons},
    fill_value =0
)

df_pokemons_tipe_and_gen['pokemon_geracao']

pokemon_tipo_1_nome,Bug,Ghost,Rock
pokemon_geracao,,,
5,75,0,0
6,0,60,54


##### 4) Faça um comparativo entre as gerações e a coluna de pokémons lendários para verificar quantos pokémons existem com velocidade acima de 85. Dica: use pivoteamento.

In [24]:
# Para buscar quantos pokémons existem com velocidade a cima de 85, optei por filtrar meu df antes de criar o pivot_table, pegando somente os que me interessa.
# Após, criei o pivot_table e apresentei a distribuição dos dados, apresentando a quantidade de pokémons com velocidade acima de 85, por geração e se é lendario ou não.

# Adicionalmente ao desafio, apresentei uma análise comparando o resultado de df_pokemons_lendarios e df_pokemons_not_lendarios com os dados da tabela pivot:
# Existe maior quantidade de pokémons rápidos não lendários que lendários, porém 85% dos pokemons lendários tem velocidade a cima de 85, quando 25% dos não lendários possuem tal velocidade.

df_veloses_pokemon = df_pokemon_complete.query('pokemon_velocidade > 85')

df_comparection_gen_and_legendary = pd.pivot_table(df_veloses_pokemon,
    index=['pokemon_geracao'],  
    columns=['pokemon_lendario'],
    aggfunc={'pokemon_geracao': np.size},
)

print(df_comparection_gen_and_legendary['pokemon_geracao'])


pokemon_lendario  False  True
pokemon_geracao              
1                    21     3
2                     9     2
3                    12     8
4                    21     5
5                    21    13
6                    10     3


##### Adicional a questão 4:

In [25]:
print('\nAdicionalmente ao desafio apresento a sequinte conclusão:')

count_pokemons_lendarios =  df_pokemon_complete.query('pokemon_lendario == True').count()['pokemon_lendario']
count_pokemons_not_lendarios =  df_pokemon_complete.query('pokemon_lendario == False').count()['pokemon_lendario']

lendario = 0
nao_lendario = 0

total_pokemons = df_pokemons_lendarios + df_pokemons_not_lendarios
total_pokemons_rapidos =  lendario + nao_lendario

percent_total = (total_pokemons_rapidos * 100) / total_pokemons

percent_lendarios = (lendario * 100) / count_pokemons_lendarios
percent_nao_lendarios = (nao_lendario * 100) / count_pokemons_not_lendarios

for i in df_veloses_lendary_pokemon['pokemon_lendario']:

    if i == True:
        lendario += 1
    else:
        nao_lendario += 1

total_pokemons = df_pokemons_lendarios + df_pokemons_not_lendarios
total_pokemons_rapidos =  lendario + nao_lendario

percent_total = (total_pokemons_rapidos * 100) / total_pokemons

percent_lendarios = (lendario * 100) / count_pokemons_lendarios
percent_nao_lendarios = (nao_lendario * 100) / count_pokemons_not_lendarios

print(f'\nO total de pokémons é de {total_pokemons}, sendo que {total_pokemons_rapidos} deles tem velocidade maior que 85.')

if lendario > nao_lendario:
    print(f'\n{lendario} são lendários e {nao_lendario} não lendários, existema mais pokémons lendários rápidos que não lendários.')
else:
    print(f'\n{nao_lendario} não são lendários e {lendario} lendários, existem mais pokémons não lendários rápidos que lendários.')

if percent_lendarios > percent_nao_lendarios:
    print(f'Porém, {round(percent_lendarios,2)} % dos lendários possuem velocidade superior a 85, quando {round(percent_nao_lendarios, 2)} % dos não lendarios possui tal velocidade.')
    print('\nConclusão: Os pokémons lendarios são, em sua maioria, mais rápidos que os não lendários.')
else:
    print(f'E {round(percent_nao_lendarios, 2)} % dos não lendários possuem velocidade superior a 85, quando {round(percent_lendarios,2)} % dos lendarios possui tal velocidade.')
    print('\nConclusão: Os não lendários são mais rapidos até proporcionalmente.')


Adicionalmente ao desafio apresento a sequinte conclusão:


NameError: name 'df_pokemons_lendarios' is not defined

##### 5) Plote um gráfico mostrando o valor somado da coluna pokemon_total por tipo de pokemon, contando apenas os pokemons que possuem defesa menor que 50.

##### 6) Faça uma comparação entre pokemons do tipo fire e do tipo grass evidenciando qual tem mais defesa no total e qual tem mais ataque no total.

In [75]:
df_fire_and_grass = df_pokemon_complete.query('pokemon_tipo_1_nome == "Fire" | pokemon_tipo_1_nome == "Grass"')

pokemons_tipe_fire_grass  = pd.pivot_table(df_fire_and_grass,
    index= ['pokemon_tipo_1_nome','pokemon_nome'],
    # columns= ['pokemon_tipo_1_nome'],
    aggfunc= {'pokemon_attack': max, 'pokemon_defense': max}
)

pokemons_tipe_fire_grass

pokemon_attack  pokemon_defense
pokemon_tipo_1_nome pokemon_nome                                          
Fire                Blaziken                          120               70
                    BlazikenMega Blaziken             160               80
                    Camerupt                          100               70
                    CameruptMega Camerupt             120              100
                    Charizard                          84               78
...                                                   ...              ...
Grass               Victreebel                        105               65
                    Vileplume                          80               85
                    Virizion                           90               72
                    Weepinbell                         90               50
                    Whimsicott                         67               85

[61 rows x 2 columns]

##### 7) Um treinador pokémon quer escolher um pokémon para a batalha. Faça uma análise para indicar o melhor pokemon possível para o embate.
##### a) O adversário possui um pokémon do tipo Grass, e que esse tipo possui desvantagem sobre pokémons do tipo Bug, Fire, Flying, Ice e Poison.
##### b) O pokémon deve ser no máximo da geração 3
##### c) O ataque do pokémon deve ser menor ou igual a 100.
##### d) A defesa deve ser maior que 30.
##### e) O pokémon pode ter um tipo secundário que não influencia na regra a.
##### f) O pokémon não pode ser lendário.